In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import json
import os
from typing import List
import geopandas as gpd
import plotly_express as px
import networkx as nx

load_dotenv()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## TODO
1. Load Aleph entities
2. Load Mapstand data
3. Add missing geometries to mapstand, drop rows that are not used
4. Merge Mapstand with Aleph entities
5. Clean it up

In [ ]:
PATH_ALEPH = os.environ.get('PATH_ALEPHDATA')
PATH_RAW = os.environ.get('PATH_RAW')

In [ ]:
def parse_json(entities: List) -> pd.DataFrame:
    '''Parses Aleph JSON data
    '''
    
    entity_list = []
    
    for entity in entities:
        data = entity.get('properties')

        for key, value in data.items():
            if isinstance(value, list):
                data.update({key: ','.join(value)})
        entity_id = {'id': entity.get('id')}
        data.update(entity_id)
        entity_list.append(data)
    
    df = pd.DataFrame(entity_list)
    return df


def load_entities(path: str, entity: str) -> pd.DataFrame:
    '''Load entities from Aleph
    (downloaded through alephclient)'''

    entities = []
    with open(f'{path}{entity}.json', 'r') as file:
        for line in file:
            entities.append(json.loads(line))

    df = parse_json(entities)
    return df

## Import Aleph entities

In [ ]:
# Import companies

companies = load_entities(PATH_ALEPH, 'companies')
companies.drop(['notes', 'summary', 'sourceUrl','publisher', 'alias', 'description', 
                'leiCode', 'parent', 'amountEur'], axis=1, inplace=True)


# Import and clean assets

assets = load_entities(PATH_ALEPH, 'assets')
assets.dropna(subset='description', inplace=True)
assets.drop(['title', 'authority', 'contractDate', 'jurisdiction', 'registrationNumber',
             'previousName', 'parent', 'leiCode', 'sourceUrl', 'publisher'], axis=1, inplace=True)

# Import ownerships

ownerships = load_entities(PATH_ALEPH, 'ownerships')

## Import geometries 

We have one geosjon with edited data on capacity and installation year, but it's not complete (I know...). So let's get the newest installed and planned windfarm dataset from MapStand


In [ ]:
# Import geometries with edited data

ms = gpd.GeoDataFrame.from_file(PATH_RAW + 'mapstand_final.geojson', geometry='geometry')
ms = ms.to_crs(4326)

# So import newest for missing geometries

msn = gpd.GeoDataFrame.from_file(PATH_RAW + 'mapstand_final_newest_installed.geojson')
msp = gpd.GeoDataFrame.from_file(PATH_RAW + 'mapstand_final_newest_planned.geojson')
wdz = gpd.GeoDataFrame.from_file(PATH_RAW + 'mapstand_final_wdz.geojson')

# Just use the items from WDZ we need

ids = ['cf34dea4-5b4a-462c-b8b5-56c1ebcebcc7', '116dd036-d8f3-49b7-b63e-625b0020993e']
wdz = wdz[wdz.mps_uuid.isin(ids)].copy()

msn = pd.concat([msn, msp])

lookup = ['HIRTSHALS HARBOUR', 
          'WP Q10 / ENECO LUCHTERDUINEN', 
          'UNITECH ZEFYROS (HYWIND DEMO/KARMOY) - METCENTRE',
          'AFLANDSHAGE', 
          'LEVENMOUTH DEMONSTRATION TURBINE',
          'BLYTH',
          'BEATRICE DEMONSTRATOR SITE',
          'EAST ANGLIA THREE'
          ]

msn = msn[msn.name.isin(lookup)][:-1].copy()
msn = msn.to_crs(4326)

# Merge the missing with the right ones

ms = pd.concat([ms, msn, wdz])

# Create selection of relevant columns

cols = ['mps_uuid', 'name', 'cost_in_million', 'year', 'capacity_mw', 'description', 'remarks',
        'number_generators',  'installation_year', 'mps_est_elevation_max_m', 'mps_est_elevation_min_m', 'geometry']

selection = ms[cols].copy()

In [ ]:
# Merge with assets

df = pd.merge(assets,
             selection,
             left_on = 'description',
             right_on='mps_uuid',
             how='outer')

# Clean column names

df = df.rename(columns={'name_x': 'name_aleph', 
                        'name_y': 'name_ms',
                        'description_y': 'description',
                        'notes': 'status'
                        })

df.drop('description_x', axis=1, inplace=True)

In [ ]:
# Clean MW

df.amount = df.amount.str.replace(' MW', '')
df.amount = df.amount.astype('float')
df['capacity_mw'] = df['capacity_mw'].fillna(df.amount)

# Clean costs
df[['amountEur', 'cost_in_million']]
df.amountEur = df.amountEur.fillna(df.cost_in_million * 1000000).astype('float')

df = df.drop(['cost_in_million', 'amount'], axis=1)

# Clean installation year

df.year = df.year.fillna(df.installation_year.astype(str).replace('.0', '') + '-01-01')

# Clean wind farms we want to leave out and mislabeled wind farm

df = df[(df.name_aleph.notna()) & (df.name_aleph != 'DIAMOND OFFSHORE WIND HOLDINGS I BV')].copy()

In [ ]:
# Add simplified status from Marina

st_dict = {'OPERATIONAL': "EXISTING", 
 'EXCLUSIVE_DEVELOPMENT_RIGHTS': "EARLY_PLANS",
 'UNDER_CONSTRUCTION': "EXISTING", 
 'PROJECT_ANNOUNCED': "EARLY_PLANS", 
 'EARLY_PLANNING': "EARLY_PLANS",
'APPLICATION_SUBMITTED': "SERIOUS_PLANS", 
'CONSENT_AUTHORISED': "SERIOUS_PLANS", 
'EXTENSION_REQUESTED': "SERIOUS_PLANS", 
'CANCELLED': "", 
'ONSHORE - OPERATIONAL': "",
'DECOMMISSIONED': "", 
'AREA_PROPOSED': "EARLY_PLANS"}

df['status_simplified'] = df.status.replace(st_dict)

In [ ]:
# Create geodataframe

gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=4326)

# Reproject so we can calculate area (WGS84/UTM zone 31N is a fine one)

gdf = gdf.to_crs(32631)
gdf['area_km'] = gdf.geometry.area / 1000000

# Write to file
gdf.to_file(PATH_ALEPH + 'analysis_v1.geojson', driver='GeoJSON')

## Analysis

We would like some basic statistics, for instance:
1. Comparisons between countries regarding output, area, number of turbines, if possible normalised with population size.
2. Temporal trends.
3. A detailed overview of company shares in wind farms. For this we would probably need Neo4J to query the ownership graph, or simplify this graph to a table. 
4. Perform linear regressions on costs of wind farms, because some data is lacking. 
5. Assess how much power is sold through PPAs

In [ ]:
# First get the population of the countries for normalization, in thousands

population = {'de': 83000,
              'gb': 68000,
              'nl': 18000,
              'be': 12000,
              'no': 5000,
              'dk': 6000}

### Capacity

In [ ]:
# Group by country and status simplified

capacity = gdf.groupby(['country', 'status_simplified']).capacity_mw.sum().reset_index()
capacity = capacity[capacity.status_simplified != ''].copy()

# Add capacity per 1000 inhabitants

capacity['cap_pop'] = capacity.capacity_mw.div(capacity.country.map(population))


In [ ]:
# Plot

fig = px.bar(capacity, 
             x='country', 
             y='cap_pop', 
             facet_col='status_simplified',
             title = 'Energy output per capita (x1000)')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.show()

### Capacity over time

In [ ]:
# Groupby year and country

c_time = gdf.groupby(['country', 'year']).capacity_mw.sum().reset_index()

In [ ]:
fig = px.area(c_time,
              x='year',
              y='capacity_mw',
              color='country',
              title='Projected megawatt production per country per year',
             )
fig.show()

## Analyse ownership structure

There are several ways to go about this. It would be great if we could (partly) automate the generation of ownership tables, using graphs. We could use neo4j for that, or networkx. A query should look something like this:
1. For each asset assign a value of 1
2. Travel on an ownership relation and multiply the weight of that relationship, e.g. 1
3. At the next node find all ownership relationships
4. Travel all relationships and multiply by the weight of that relationship, e.g. .5


One of the problems with ownership is that we have some ranges and non-precise values (e.g. 75+). A solution is to define lower and upper values and convert them to weights. So this means we would create a few extra columns (percentage_lower_bound, percentage_upper_bound) and convert them to proper percentages so we can easily use them for multiplication. 

For now we have to assume that a missing percentage is 100 percent. That will often be te case, but we have to go through the Aleph data again one time to fill in the missing percentages.

One promising approach is to use Dijkstra's Algorithm, the shortest path, between companies (source) and assets (targets). Because we're dealing with a directed graph, this should omit any detours because companies have joint ventures in other projects. 

In [ ]:
# Clean percentages

ownerships.percentage = ownerships.percentage.str.replace('+', '-100')
ownerships.percentage.fillna('100', inplace=True)
ownerships.percentage = ownerships.percentage.astype('str')

# Add columns for lower and upper bound

ownerships['perc_lower'] = ownerships.percentage.apply(lambda x: float(x.split('-')[0]) / 100 if '-' in x else float(x) / 100)
ownerships['perc_higher'] = ownerships.percentage.apply(lambda x: float(x.split('-')[1]) / 100 if '-' in x else float(x) / 100)

In [ ]:
# Create directed graph

G = nx.DiGraph()

In [ ]:
for i, row in assets.iterrows():
    G.add_node(row.id,
               name=row['name'],
               status=row.notes,
               country=row.country, 
               costs=row.amountEur, 
               mps_uuid=row.description,
               aleph_url=row.alephUrl
               )
    
for i, row in companies.iterrows():
    G.add_node(row.id,
               name=row['name'],
               country=row.jurisdiction,
               registration=row.registrationNumber,
               source_url=row.publisherUrl,
               aleph_url=row.alephUrl
               )
    
for i, row in ownerships.iterrows():
    G.add_edge(row.owner,
               row.asset,
               weight_lower=row.perc_lower,
               weight_upper=row.perc_higher,
               percentage=row.percentage,
               source=row.publisherUrl,
               aleph_url=row.alephUrl,
               id=id,
               description=row.description
               )



In [ ]:
# Get source, target and use Dijkstra's algorithm to get the nodes and relationships inbetween

source = 'IBERDROLA SA'
target = 'BALTIC EAGLE'

s = [n for n, v in nx.get_node_attributes(G, 'name').items() if v == source][0]
t = [n for n, v in nx.get_node_attributes(G, 'name').items() if v == target][0]

shortest_path = [x for x in nx.dijkstra_path(G, s, t)]

window_size = 2
weight = 1

for i in range(len(shortest_path) - window_size + 1):
    node1 = shortest_path[i: i + window_size][0]
    node2 = shortest_path[i: i + window_size][1]
    e = list(G.edges([node1, node2], data=True))
    weight *= e[0][2]['weight_lower']
    
    # Print outcome
    print(f"Node weight = {e[0][2]['weight_lower']} and cumulative weight = {weight}")


## TODO:
1. Better output of shortest path, weights and cumulative weight, so we can check the results
2. Add logic to run Dijkstra's Algorithm against all companies (sources) and assets (targets)
3. Check results:
    a. are all relationships in the right direction?
    b. do all asset ownerships add up to 1? 
4. Create a nice table with ownerships